# **Создание и обучение модели**

**Подключение библиотек**

In [146]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import joblib

**Импорт dataset'a:**

In [4]:
HeartAttack_df = pd.read_csv('/content/heart.csv', delimiter = ',')

**Подготовка данных:**

In [6]:
X = HeartAttack_df.drop('output', axis = 1) #Матрица признаков
y = HeartAttack_df['output'] #Вектор ответов

**Разделим выборку на обучающую и тестовую:**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# **Создание моделей с разными методами**

**Метод логистической регрессии**

Создадим модель, указав при этом solver:

In [139]:
LR_clf = LogisticRegression(solver = 'saga')

Опишем сетку по которой будем искать:

In [140]:
LR_param_grid = {
    'C': np.arange(0.0001, 100),
    'penalty': ['l2'],
    'solver': ['lbfgs'],
    'max_iter': [300]
}

Создадим объект GridSearchCV:

In [141]:
LR_search = GridSearchCV(LR_clf, LR_param_grid, n_jobs=-1, cv=5, refit=True, scoring='accuracy')

Запустим поиск:

In [142]:
LR_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=5, estimator=LogisticRegression(solver='saga'), n_jobs=-1,
             param_grid={'C': array([1.00000e-04, 1.00010e+00, 2.00010e+00, 3.00010e+00, 4.00010e+00,
       5.00010e+00, 6.00010e+00, 7.00010e+00, 8.00010e+00, 9.00010e+00,
       1.00001e+01, 1.10001e+01, 1.20001e+01, 1.30001e+01, 1.40001e+01,
       1.50001e+01, 1.60001e+01, 1.70001e+01, 1.80001e+01, 1.90001e+01,
       2.00001e+01, 2.1...
       7.50001e+01, 7.60001e+01, 7.70001e+01, 7.80001e+01, 7.90001e+01,
       8.00001e+01, 8.10001e+01, 8.20001e+01, 8.30001e+01, 8.40001e+01,
       8.50001e+01, 8.60001e+01, 8.70001e+01, 8.80001e+01, 8.90001e+01,
       9.00001e+01, 9.10001e+01, 9.20001e+01, 9.30001e+01, 9.40001e+01,
       9.50001e+01, 9.60001e+01, 9.70001e+01, 9.80001e+01, 9.90001e+01]),
                         'max_iter': [300], 'penalty': ['l2'],
                         'solver': ['lbfgs']},
             scoring='accuracy')

Выведем наилучшие параметры:

In [143]:
print(LR_search.best_params_)

{'C': 1.0001, 'max_iter': 300, 'penalty': 'l2', 'solver': 'lbfgs'}


Найдем точность полученной модели (с наилучшими параметрами), построенной на методе логистической регрессии:

In [144]:
LR_best_model = LR_search.best_estimator_
LR_accuracyScore = accuracy_score(y_test, LR_search.best_estimator_.predict(X_test))
print(f"Точность модели: {LR_accuracyScore}")

Точность модели: 0.8688524590163934


**Метод ближайших соседей**

Создадим модель:


In [31]:
KNC_clf = KNeighborsClassifier()

Опишем сетку по которой будем искать:

In [32]:
KNC_params_grid = {
    'n_neighbors': np.arange(1, 20),
    'weights': ['uniform', 'distance'],
    'metric': ['manhattan', 'euclidean'],
}

Создадим объект GridSearchCV:

In [33]:
KNC_search = GridSearchCV(KNC_clf, KNC_params_grid, cv=5, scoring='accuracy', n_jobs=-1)

Запустим поиск:

In [34]:
KNC_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['manhattan', 'euclidean'],
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19]),
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

Выведем наилучшие параметры:

In [35]:
print(KNC_search.best_params_)

{'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}


Найдем точность полученной модели (с наилучшими параметрами), построенной на методе ближайших соседей:

In [36]:
KNC_accuracyScore = accuracy_score(y_test, KNC_search.best_estimator_.predict(X_test))
print(f"Точность модели: {KNC_accuracyScore}")

Точность модели: 0.7049180327868853


**Метод "Деревья решений"**

Создадим модель:

In [40]:
DT_clf = DecisionTreeClassifier()

Опишем сетку по которой будем искать:

In [45]:
DT_params_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 50),
    'min_samples_leaf': np.arange(1, 50),
}

Создадим объект GridSearchCV:

In [47]:
DT_search = GridSearchCV(DT_clf, DT_params_grid, cv=5, scoring='accuracy', n_jobs=-1)

Запустим поиск:

In [48]:
DT_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                         'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                         'splitter': ['best', 'random']},
             scoring='accuracy')

Выведем наилучшие параметры:

In [49]:
print(DT_search.best_params_)

{'criterion': 'log_loss', 'max_depth': 40, 'min_samples_leaf': 3, 'splitter': 'random'}


Найдем точность полученной модели (с наилучшими параметрами), построенной на методе "Деревья решений":

In [50]:
DT_accuracyScore = accuracy_score(y_test, DT_search.best_estimator_.predict(X_test))
print(f"Точность модели:{DT_accuracyScore}")

Точность модели:0.7868852459016393


**Метод опорных векторов**

Создадим модель:

In [54]:
svc_clf = SVC(random_state = 0)

Опишем сетку по которой будем искать:

In [128]:
svc_params_grid = {
    'C': np.arange(100, 120),
    'kernel': ['linear'],
    'gamma': ['scale'],
    'degree': [1]
}

Создадим объект GridSearchCV:

In [129]:
svc_search = GridSearchCV(svc_clf, svc_params_grid, cv=5, scoring='accuracy', n_jobs=-1)

Запустим поиск:

In [130]:
svc_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(random_state=0), n_jobs=-1,
             param_grid={'C': array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112,
       113, 114, 115, 116, 117, 118, 119]),
                         'degree': [1], 'gamma': ['scale'],
                         'kernel': ['linear']},
             scoring='accuracy')

Выведем наилучшие параметры:

In [131]:
print(svc_search.best_params_)

{'C': 102, 'degree': 1, 'gamma': 'scale', 'kernel': 'linear'}


Найдем точность полученной модели (с наилучшими параметрами), построенной на методе опорных векторов:

In [132]:
svc_accuracyScore = accuracy_score(y_test, svc_search.best_estimator_.predict(X_test))
print(f"Точность модели:{svc_accuracyScore}")

Точность модели:0.8360655737704918


**Метод случайного леса**

Создадим модель:

In [74]:
RF_clf = RandomForestClassifier()

Опишем сетку по которой будем искать:

In [89]:
RF_params_grid = {
    'n_estimators': np.arange(1, 101, 5),
    'max_depth': np.arange(1, 41, 10),
    'min_samples_split': np.arange(1, 11, 2),
}

Создадим объект GridSearchCV:

In [90]:
RF_search = GridSearchCV(RF_clf, RF_params_grid, cv=5, scoring='accuracy', n_jobs=-1)

Запустим поиск:

In [91]:
RF_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
400 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
400 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 340, in fit
    self._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 600, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/u

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': array([ 1, 11, 21, 31]),
                         'min_samples_split': array([1, 3, 5, 7, 9]),
                         'n_estimators': array([ 1,  6, 11, 16, 21, 26, 31, 36, 41, 46, 51, 56, 61, 66, 71, 76, 81,
       86, 91, 96])},
             scoring='accuracy')

Выведем наилучшие параметры:

In [92]:
print(RF_search.best_params_)

{'max_depth': 31, 'min_samples_split': 9, 'n_estimators': 11}


Найдем точность полученной модели (с наилучшими параметрами), построенной на методе случайного леса:

In [93]:
RF_accuracyScore = accuracy_score(y_test, RF_search.best_estimator_.predict(X_test))
print(f"Точность модели:{RF_accuracyScore}")

Точность модели:0.8524590163934426


# **Итог**

После того, как мы создали несколько моделей с различными методами машинного обучения, такими как:
1. Логистической регрессии
2. Ближайших соседей
3. "Деревья решений"
4. Опорных векторов
5. Случайного леса

мы получили, что наилучшая модель, с точностью модели: 0.8688524590163934
является модель построенная с помощью метода **логистической регресии**
Сохраним ее в отдельный файл, чтобы позже внести в проект.

In [147]:
best_model = LR_best_model
joblib.dump(best_model, 'HeartAttack_model.joblib')

['HeartAttack_model.joblib']